In [1]:
#HYPER-PARAMETERS
k           = 7
DIMR        = 250
PCA_flag    = False
LCN_flag    = False
num_epoch   = 270
num_hid     = DIMR
batch_sz    = 1000



import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from IPython import display
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import random
import torchvision
from tqdm import trange
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


In [2]:


%matplotlib inline


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device: ', device)
# set_random_seeds(seed_value=0, device=device)



root = './data'
if not os.path.exists(root):
    os.mkdir(root)
    
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

#validation split
split_coeff = 0.2
train_sampler = SubsetRandomSampler(range(int(split_coeff*len(train_set)), len(train_set)))
valid_sampler = SubsetRandomSampler(range(int(split_coeff*len(train_set))))

val_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_sz,
                 sampler=valid_sampler)

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_sz,
                 sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_sz,
                shuffle=False)

print('total trainning batch number: {}'.format(len(train_loader)))
print('total val batch number: {}'.format(len(val_loader)))
print('total testing batch number: {}'.format(len(test_loader)))

device:  cpu
Processing...
Done!
total trainning batch number: 48
total val batch number: 12
total testing batch number: 10


In [3]:
img_side = np.array(train_set[0][0]).shape[1]
print('img_side: ',img_side  )
num_classes = 10 #for mnist

img_side:  28


In [5]:


class cccml(nn.Module):
    def __init__ (self, dim_reduce, img_side,  num_classes, batch_size, num_channels=1, nkerns=[10,10], poolsize=(2, 2), f0 = 5, f1 = 3):
        super().__init__()
#         self.D =  int(np.sqrt(num_dims / num_channels))
        self.D = img_side
        self.batch_size = batch_size
        self.dim_reduce = dim_reduce
        self.num_classes = num_classes
        
        
        numpy_rng=np.random.RandomState(1234)
        
        self.image_shape0=[batch_size, num_channels, self.D, self.D]
        self.filter_shape0=(nkerns[0], num_channels, f0, f0) #TODO 
        img_out_size0= (self.D-f0+1)/poolsize[0]
        
        self.image_shape1=[batch_size, nkerns[0], img_out_size0, img_out_size0]
        self.filter_shape1=(nkerns[1], nkerns[0], f1, f1) #TODO 
        img_out_size1= (img_out_size0-f1+1)/poolsize[0]
        print('Layer0: Filter size %d, image size out %d' % (f0, img_out_size0))
        print('Layer1: Filter size %d, image size out %d' % (f1, img_out_size1))
       
        convL1_dim = img_out_size1
        num_convH = nkerns[-1]*convL1_dim*convL1_dim
        print(num_convH)
        print(convL1_dim)
        
        
        self.stage1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channels, out_channels=10, kernel_size=f0),
            nn.BatchNorm2d(10),
            nn.MaxPool2d(kernel_size=2), 
            nn.Tanh())
        self.stage2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=100, kernel_size=f1),
            nn.BatchNorm2d(100),
            nn.MaxPool2d(kernel_size=2), 
            nn.Tanh())
        
        self.fc = nn.Sequential(
            nn.Linear(2500, self.dim_reduce, bias=False))

        
    def forward(self, X, num_train=None):
        X = self.stage1(X)
        X = self.stage2(X)
#         print(np.array(X).shape)
        X = self.fc(X.view(-1, 2500))

        return X

In [6]:
k_near = 2
p_metr = 2

class CKNN_loss(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
           
    @staticmethod
    def get_priors(inp, target):
        priors = {}
        n = target.size()[0]
        d = {x: list(target.cpu().numpy()).count(x) for x in target.cpu().numpy()}
#         a, b = d.keys(), d.values()
        for i in range(len(d.keys())):
            priors[str(torch.tensor(list(d.keys())[i]))] = torch.tensor(np.float64(list(d.values())[i]), requires_grad=True)
        return priors
    
    @staticmethod
    def get_distances(inp, target):
        dist_dict = {}
        for t in torch.unique(target):
            dist_dict[str(t)] = []
        for t in torch.unique(target):
            for i in torch.arange(len(target)):
                if target[i].cpu().numpy() == t.cpu().numpy():
                    dist_dict[str(t)].append(torch.norm(inp[-1][0] - inp[i][0], p=p_metr))
        for t in torch.unique(target):
            #print(dist_dict[str(t)], 123231)
            dist_dict[str(t)] = torch.tensor(dist_dict[str(t)], requires_grad=True)
            #print(dist_dict[str(t)])
            dist_dict[str(t)] = dist_dict[str(t)].topk(k=k_near, largest=False)[0]
        return dist_dict
    
    @staticmethod
    def get_sigma(dist_dict):
        return torch.std(torch.flatten(torch.stack(list(dist_dict.values()))))
    
    @staticmethod
    def get_likelihood(near_dict_dist, sigma):
        likelihood_dict = {}
        for key in near_dict_dist.keys():
            curr_list = near_dict_dist[key]
            likelihood_dict[key] = torch.exp(-torch.sum(curr_list**2) / sigma)
        return likelihood_dict

    @staticmethod
    def get_posteriors(priors, likelihoods):
        #print(priors.keys())
        posteriors  = {key: priors[str(key)] * likelihoods[str(key)] for key in priors.keys()}
        return posteriors
    
    @staticmethod
    def get_term_of_loss(inp, target, i):
        priors     = CKNN_loss.get_priors(inp, target)
        distances  = CKNN_loss.get_distances(inp, target)
        sigma      = CKNN_loss.get_sigma(distances)
        likelih    = CKNN_loss.get_likelihood(distances, sigma)
        posteriors = CKNN_loss.get_posteriors(priors, likelih)
        return -posteriors[str(target[i])]
    
    @staticmethod
    def forward(inp, target):
#         print(target.size())
        res = torch.tensor(0.)
#         for i in torch.arange(target.size()[0]):
        for i in torch.arange(torch.tensor(10)):

            inp1 = torch.cat((torch.index_select(inp, 1, Variable(torch.LongTensor(list(range(i)) + list(range(i+1, DIMR))))),
                              torch.index_select(inp, 1, Variable(torch.LongTensor([i])))), dim=1)
            target1 = torch.index_select(target, 0, Variable(torch.LongTensor(list(range(i)) + list(range(i+1, batch_sz)))))


            torch.add(res, CKNN_loss.get_term_of_loss(inp, target, target[i]))
        return res

In [11]:
from IPython import display
from tqdm import trange
import os
def get_score(model, dataloader, device='cpu'):
    ac = CKNN_loss()
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)  
            y = y.to(device)  
            dataloader_neg_posterior += ac(x, y)
    return dataloader_neg_posterior/len(dataloader)
def train(model, optimizer, criterion, train_loader, val_loader,device, epochs_n=100):  
    model.to(device)

    learning_curve = [np.nan] * epochs_n
    train_accuracy_curve = [np.nan] * epochs_n
    val_accuracy_curve = [np.nan] * epochs_n
    max_val_accuracy = 0
    max_val_accuracy_epoch = 0
    train_size = len(train_loader)
    batch_size=16
    batches_n = (train_size - 1) // batch_size + 1

    for epoch in range(epochs_n):
        model.train()

        learning_curve[epoch] = 0
        for i, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            prediction = model(x).requires_grad_(True)
            
            loss = criterion(prediction, y)
            loss = Variable(loss, requires_grad = True)
            learning_curve[epoch] += loss.item()
            loss.backward()
            optimizer.step()
            print(loss.item())
            print(i)

        display.clear_output(wait=True)
        f, axes = plt.subplots(1, 2, figsize=(15, 3))

        learning_curve[epoch] /= batches_n
        axes[0].plot(learning_curve)

        model.eval()
        train_accuracy_curve[epoch] = get_accuracy(model, train_loader, device)
        val_accuracy_curve[epoch] = get_accuracy(model, val_loader, device)
        
        val_accuracy = val_accuracy_curve[epoch]
        if val_accuracy > max_val_accuracy:
            max_val_accuracy = val_accuracy
            max_val_accuracy_epoch = epoch
            model_dir = os.getcwd()+"/best_model"
            torch.save(model, model_dir)
        
        axes[1].set_title('Train {:.4f}, val {:.4f}, max val {:.4f} at {}'.format(
            train_accuracy_curve[epoch], val_accuracy, max_val_accuracy, max_val_accuracy_epoch))
        axes[1].plot(train_accuracy_curve)
        axes[1].plot(val_accuracy_curve)

        plt.tight_layout()
        plt.show()

In [12]:
from torchsummary import summary
model = cccml(dim_reduce=num_hid, img_side=img_side, num_classes=num_classes, batch_size=batch_sz).to(device)
optimizer = torch.optim.SGD(model.parameters(),  lr=0.005, momentum=0.9)
criterion = CKNN_loss() ##
#         knn = KNN()
# hyper_params=[num_dims, num_train_cases, num_test_cases, num_hid, num_classes, \
#                 num_epoch, epsilon, batch_sz, k, model_type, weightcost]
#         knn_acc, cknn_acc, acc3, acc4 = run(train_set, valid_set, test_set, knn, hyper_params)
#         save.append([knn_acc, cknn_acc, acc3, acc4])
summary(model, (1, 28, 28), batch_sz)

Layer0: Filter size 5, image size out 12
Layer1: Filter size 3, image size out 5
250.0
5.0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [1000, 10, 24, 24]             260
       BatchNorm2d-2         [1000, 10, 24, 24]              20
         MaxPool2d-3         [1000, 10, 12, 12]               0
              Tanh-4         [1000, 10, 12, 12]               0
            Conv2d-5        [1000, 100, 10, 10]           9,100
       BatchNorm2d-6        [1000, 100, 10, 10]             200
         MaxPool2d-7          [1000, 100, 5, 5]               0
              Tanh-8          [1000, 100, 5, 5]               0
            Linear-9                [1000, 250]         625,000
Total params: 634,580
Trainable params: 634,580
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 2.99
Forward/backward pass size (MB): 302.51
Param

In [ ]:
train(model, optimizer, criterion, train_loader, val_loader,device=device, epochs_n=100)